<a href="https://colab.research.google.com/github/AlexWhite-USDA/NDII_33km/blob/main/ASRS24_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 264-2 Interactive Vegetation Index Mapping and Time Series Extraction Using the Google Earth Engine API for Python
# https://scisoc.confex.com/scisoc/2024am/meetingapp.cgi/Paper/158750
# Tri Societies 2024 ASRS symposium
# William (Alex) White
# November 12, 2024

In [ ]:
import folium

In [ ]:
# Set lat, lon and create folium map object
lat = 29.42
lon = -98.484
folium.Map([lat,lon])

In [ ]:
# Add zoom argument, then add marker, and m object
m = folium.Map([lat,lon], zoom_start=17)
folium.Marker([lat, lon]).add_to(m)
m

In [ ]:
# Import ee, authenticate, and initialize (click through account access screens)
import ee
ee.Authenticate()
ee.Initialize(project='asrs24')

In [ ]:
# Define folium method (credit: Qiusheng Wu)
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=qBFjMAQkfK7A
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Load NAIP
# https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ
naip = ee.ImageCollection('USDA/NAIP/DOQQ') \
.filterDate('2022-01-01', '2024-12-31') \
.select(['R', 'G', 'B'])

m.add_ee_layer(naip,{'min': 0.0, 'max': 255.0},'NAIP basemap')
m

In [ ]:
# Set coordinates to farmland, add a layer control panel to the map
lat = 29.35
lon = -98.8

m = folium.Map([lat,lon], zoom_start = 10)
folium.Marker([lat,lon]).add_to(m)
m.add_ee_layer(naip,{'min': 0.0, 'max': 255.0},'NAIP basemap')
m.add_child(folium.LayerControl())

In [ ]:
# Load and filter Landsat 9 collection – note reversed order of lon, lat
l9sr = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
.filterDate('2024-10-15','2024-10-31') \
.filterBounds(ee.Geometry.Point([lon,lat]))

In [ ]:
# Try different dates, and get JSON info
l9sr.getInfo()

In [ ]:
# Get collection size info
l9sr.size().getInfo()

In [ ]:
# Get first image date
l9sr.first().getInfo()['properties']['DATE_ACQUIRED']

In [ ]:
# Add Landsat collection to map, with vis parameters
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

dataset = l9sr.map(apply_scale_factors)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
    }

m.add_ee_layer(dataset,visualization,'Landsat')

In [ ]:
# Cloud mask – map a function to entire collection (open link)
# https://developers.google.com/earth-engine/landsat_c1_to_c2#colab-python_1

def cloudMask(scene):
  qa_mask = scene.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  return scene.updateMask(qa_mask)

l9sr_masked = l9sr.map(cloudMask)

In [ ]:
# Run all map commands in 1 cell, view map full-screen and toggle between layers
m = folium.Map([lat,lon], zoom_start=17)
folium.Marker([lat, lon]).add_to(m)
m.add_ee_layer(naip,{'min': 0.0, 'max': 255.0},'NAIP basemap')
m.add_ee_layer(l9sr.map(apply_scale_factors),visualization,'Landsat')
m.add_ee_layer(l9sr_masked.map(apply_scale_factors),visualization,'Landsat masked')
m.add_child(folium.LayerControl())

In [ ]:
#	Edit date range to 10-01-2024 – 10-31-2024, run cells again, and show cloud masked result is mosaic of the 2 images

In [ ]:
# NDII function and vis params
# Note that .select('NDII') removes other bands

def ndiiL9(scene):
  ndii = scene.normalizedDifference(['SR_B5', 'SR_B6']).select([0], ['NDII'])
  return scene.addBands(ndii)

l9sr_ndii = l9sr_masked.map(ndiiL9).select('NDII')

vis_params = {
  'min': 0,
  'max': 1.0,
  'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

In [ ]:
# Show new band in JSON
l9sr_ndii.getInfo()

In [ ]:
# Edit cell with map commands, to add ndii layer to map and view results… paste:
# m.add_ee_layer(l9sr_ndii, vis_params, 'Landsat NDII')

In [ ]:
# Demo full-fledged tool
# https://github.com/AlexWhite-USDA/NDII_33km/